In [0]:
import numpy as np
import pandas as pd
import random
from scipy.spatial import distance

In [0]:
def greedy_k_center(data, k):
  data_copy = data.copy()
  n = len(data.index)
  facilities = []
  facilities.append(random.randint(0, n-1))
  data_copy.drop(facilities[0])
  lamb_list = {}
  for l in range(1,k):
    client_list = {}
    for i in range(len(data_copy.index)):
      min_dist = float('inf')
      for f in facilities:
        min_dist = min(min_dist, distance.euclidean(data_copy.loc[i][0],data_copy.loc[f][0]))
      client_list[i] = min_dist
    new_facility = max(client_list, key = lambda j: client_list[j])
    facilities.append(new_facility)
    data_copy.drop([new_facility])
  for j in range(len(data_copy.index)):
    min_dist_lamb = float('inf')
    for f in facilities:
      min_dist_lamb = min(min_dist_lamb, distance.euclidean(data_copy.loc[j][0],data_copy.loc[f][0]))
    lamb_list[j] = min_dist_lamb
  lamb = min(lamb_list)
  return facilities, lamb

    

In [0]:
df = pd.DataFrame()
vectors = [np.random.random_sample((10,)) for i in range(100)]
df["Doc2Vec"] = vectors
authors_l = ["Shakespeare", "Austen", "Dickens", "Irina"]
authors = [np.random.choice(authors_l) for i in range(100)]
df["Author"] = authors

In [0]:
import pickle

In [0]:
df2 = pd.read_pickle("./train.pkl")

In [0]:
df2

<function pandas.io.pickle.read_pickle>

In [9]:
greedy_k_center(df, 15)

([32, 56, 60, 10, 66, 0, 8, 73, 15, 77, 69, 24, 99, 55, 72], 0)

In [0]:
def greedy_k_center2(data, k):
    n = len(data.index)
    data['facility'] = 0
    facilities = []
    first = random.randint(0, n-1)
    facilities.append(first)
    data['facility'][first] = 1
    lamb_list = {}
    
    for l in range(k-1):
        client_list = {}
        for i in range(len(data.index)):
            if data['facility'][i] != 1:    
                min_dist = float('inf')
                for f in facilities:
                    min_dist = min(min_dist, distance.euclidean(data.loc[i]['Vector'], data.loc[f]['Vector']))
                client_list[i] = min_dist
        new_facility = max(client_list, key = lambda j: client_list[j])
        facilities.append(new_facility)
        data['facility'][new_facility] = 1
    return facilities
        

In [0]:
def greedy_lambda(F, F_distances, lamb):
    removed = []
    Fcopy = F.copy()
    Fcopy1 = F.copy()
    for i in Fcopy:
        if i not in removed:
            for j in Fcopy:
                if j not in removed:
                    if (i == j):
                        pass
                    elif (F_distances[i,j] > 2*lamb):
                        pass
                    else:
        
                        Fcopy1.remove(j)
                        removed.append(j)
    return removed

In [0]:
def update_distance(row, new_facility):
    new_dist = distance.euclidean(row['Vector'], df.loc[new_facility]['Vector'])
    current_dist = row['min_dist']
    if (new_dist < current_dist):
        row['min_dist'] = new_dist
        row['assignment'] = new_facility
    return row

def max_index(a):
    max_val = -1
    max_i = -1
    for i in range(len(a)):
        if (a[i] > max_val):
            max_val = a[i]
            max_i = i
    return max_i
        

In [0]:
def greedy_k_center3(data, k):
    n = len(data.index)
    data['facility'] = 0
    data['min_dist'] = float('inf')
    data['assignment'] = -1
    facilities = []
    first = random.randint(0, n-1)
    facilities.append(first)
    data['facility'][first] = 1
    data = data.apply(update_distance, axis = 1, new_facility = first)
    for l in range(k-1):
        next_f = max_index(data['min_dist'])
        facilities.append(next_f)
        data['facility'][next_f] = 1
        data = data.apply(update_distance, axis=1, new_facility = next_f)
    lamb = max(data['min_dist'])
    return data, lamb
        

In [0]:
output = greedy_k_center2(df, 25)
print("lambda: ", output[1])
output[0].head()